In [75]:
from cflow.cflow_parser import *

In [8]:
project_cflow_results = parse_from_build_log("build_kernel_log")

In [9]:
# Building kernel index

kernel_files = [f for f in project_cflow_results.keys() if "kernel" in f]

# Avoiding duplicating definitions
excluded_files = [
    "/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/floppy.c",
    "/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/unix/phantom.c",
    "/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/init.h", # unix hal > this header
]

kernel_index = Index()

for f in kernel_files:
    if f in excluded_files:
        continue
        
    t = project_cflow_results[f]
    parse_cflow_output(kernel_index, t.decode("utf-8"))


          old: FRESULT dir_read (DIR *dj) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/ff.c:980
          new: size_t dir_read (struct uufile *f, void *dest, size_t bytes) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/unix/dir.c:270


In [11]:
# Building VM index

vm_files = [f for f in project_cflow_results.keys() if "/vm/" in f]

vm_index = Index()

for f in vm_files:
    if f in excluded_files:
        continue
        
    t = project_cflow_results[f]
    parse_cflow_output(vm_index, t.decode("utf-8"))

          old: void hal_cpu_reset_real (void) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/init.h:128
          new: void hal_cpu_reset_real () at /home/anton/Sandbox/PhantomOS/test/phantomuserland/phantom/vm/unix_hal.c:408


In [12]:
kernel_index.get_callers("phantom_phys_alloc_page()")

['phantom_phys_alloc_init()',
 'hal_alloc_vaddress()',
 'hal_alloc_phys_page()',
 'alloc_ldt_selector()']

In [13]:
kernel_index.get_callers("hal_alloc_phys_page()")

['driver_virtio_net_write()',
 'page_fault_read()',
 'provide_buffers()',
 'page_fault_snap_aid()',
 'page_fault_write()']

In [15]:
# Let's try to parse a new init routine and get its leaf nodes inside kernel index
# XXX : This file is from main repo, not from test

init_routines_new_cmd = generate_cflow_command("/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/isomem/main.c" , 'gcc -c -o main.o -W -Wall -Wno-unused-parameter -Wno-sign-compare -m64 -O0 -g -MD  -I/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/isomem -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode_libc/libc -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode_libc/libc -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode_libc/spec/x86_64/libc -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode_libc/spec/x86_64/libc -I/home/anton/Sandbox/PhantomOS/phantomuserland/include -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode-amd64 -I/home/anton/Sandbox/PhantomOS/phantomuserland/include -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode-amd64 -nostdinc -std=gnu89 -DKERNEL -DARCH_genode-amd64=1 -DBOARD_amd64-pc=1 -DBOARD=amd64-pc -include /home/anton/Sandbox/PhantomOS/phantomuserland/include/kernel/config.h -include /home/anton/Sandbox/PhantomOS/phantomuserland/include/genode-amd64/arch/arch-config.h -include /home/anton/Sandbox/PhantomOS/phantomuserland/include/genode-amd64/arch/board-amd64-pc-config.h  -fno-stack-protector -DPHANTOM_GENODE main.c')
init_routines_index = Index()
parse_cflow_output(init_routines_index, run_cflow(init_routines_new_cmd).decode("utf-8"))

In [19]:
# Let's get leaf nodes

main_subgraph = init_routines_index.get_subgraph(["main()"], lambda x: False, [])
main_leafs = [key for key, nodes in main_subgraph.items() if len(nodes) == 0]

sorted(main_leafs)

['activate_all_threads()',
 'amap_init()',
 'arch_threads_init()',
 'board_init_kernel_timer()',
 'dpc_init()',
 'driver_genode_disk_init()',
 'driver_genode_disk_probe()',
 'exit()',
 'hal_cpu_reset_real()',
 'hal_init()',
 'hal_init_physmem_alloc()',
 'hal_init_physmem_alloc_thread()',
 'hal_page_control()',
 'hal_sleep_msec()',
 'hal_sti()',
 'heap_init_mutex()',
 'load_classes_module()',
 'panic()',
 'partition_pager_init()',
 'phantom_init_part_pool()',
 'phantom_paging_init()',
 'phantom_process_boot_options()',
 'phantom_shutdown()',
 'phantom_snap_threads_interlock_init()',
 'phantom_timed_call_init()',
 'printf()',
 'pvm_alloc_threaded_init()',
 'pvm_root_init()',
 'run_init_functions()',
 'run_test()',
 'select_phantom_partition()',
 'snprintf()',
 'strcmp()',
 'strncpy()',
 'svn_version()',
 'vm_enable_regular_snaps()',
 'vm_map_init()']

In [23]:
# Check that there is no unknown nodes except new ones

set(main_leafs) - set(kernel_index.store.keys())

{'driver_genode_disk_init()', 'driver_genode_disk_probe()'}

In [25]:
# Use those leaf nodes as starting nodes for subgraph in kernel index

kernel_main_subgraph = kernel_index.get_subgraph(set(main_leafs).intersection(set(kernel_index.store.keys())), lambda x : False, [])
kernel_main_leaves = [key for key, nodes in kernel_main_subgraph.items() if len(nodes) == 0]

sorted(kernel_main_leaves)

['GET_CPU_ID()',
 'MD5_Final()',
 'MD5_Init()',
 'MD5_Update()',
 'XTeaCryptStr()',
 'XTeaDecryptStr()',
 '__builtin_constant_p()',
 '__builtin_popcount()',
 '__builtin_va_end()',
 '__builtin_va_start()',
 '__inet_itoa()',
 '__inet_ntoa()',
 '__sync_add_and_fetch()',
 '__volatile()',
 '__volatile__()',
 '_empty()',
 '_ff()',
 '_ff_fail()',
 'acpi_powerdown()',
 'acpi_reboot()',
 'activate_all_threads()',
 'add_to_queue()',
 'amap_check()',
 'amap_check_modify()',
 'amap_destroy()',
 'amap_init()',
 'amap_iterate_flags()',
 'amap_modify()',
 'arch_get_fault_address()',
 'arch_get_frame_pointer()',
 'arch_get_rtc_delta()',
 'arch_get_tick_rate()',
 'arch_get_time_delta()',
 'arch_is_object_land_access_enabled()',
 'atoi()',
 'atol()',
 'atomic_add()',
 'board_init_kernel_timer()',
 'bulk_seek_f()',
 'bzero()',
 'calloc()',
 'checkLeaveMutex()',
 'checkRange()',
 'count()',
 'create_pool()',
 'dbg_add_command()',
 'decode_base64()',
 'destroy_pool()',
 'disk_page_cache_modified()',
 'disk

In [26]:
# Looks kind of OK. But how did we end up being in udp and wtty calls as well as usys_open/read/stat/close?

kernel_index.get_callers("wtty_init()")

['get_kolibri_state()', 'do_test_wtty()', 'es1370_interrupt()']

In [27]:
kernel_index.get_callers("do_test_wtty()")

['run_test()']

In [29]:
kernel_index.get_callees("run_test()")

['do_test_ports()',
 'do_test_rectangles()',
 'do_test_crypt()',
 'panic()',
 'do_test_udp_send()',
 'do_test_amap()',
 'get_current_tid()',
 'do_test_sem()',
 'do_test_dpc()',
 'report()',
 'svn_version()',
 'do_test_physmem()',
 'do_test_physalloc_gen()',
 'do_test_hdir()',
 'setjmp_machdep()',
 'do_test_cbuf()',
 'do_test_resolver()',
 'do_test_many_threads()',
 'fhandler_f()',
 'do_test_pool()',
 'do_test_misc()',
 'do_test_absname()',
 'do_test_malloc()',
 'do_test_threads()',
 'do_test_wtty()',
 'do_test_tftp()',
 'strcmp()',
 'do_test_udp_syslog()',
 'printf()',
 'do_test_video()',
 'do_test_timed_call()',
 'do_test_tcp_connect()']

In [35]:
# Ok, that is reasonable :) 
# Let's exclude "run_test()" and see what results we will get

kernel_main_subgraph = kernel_index.get_subgraph(set(main_leafs).intersection(set(kernel_index.store.keys())), lambda x : x in ["run_test()"], ["run_test()"])
kernel_main_leaves = [key for key, nodes in kernel_main_subgraph.items() if len(nodes) == 0]

sorted(kernel_main_leaves)

['GET_CPU_ID()',
 '__builtin_constant_p()',
 '__builtin_popcount()',
 '__builtin_va_end()',
 '__builtin_va_start()',
 '__volatile()',
 '__volatile__()',
 'acpi_powerdown()',
 'acpi_reboot()',
 'activate_all_threads()',
 'add_to_queue()',
 'amap_destroy()',
 'amap_init()',
 'amap_iterate_flags()',
 'amap_modify()',
 'arch_get_fault_address()',
 'arch_get_frame_pointer()',
 'arch_get_rtc_delta()',
 'arch_get_tick_rate()',
 'arch_get_time_delta()',
 'arch_is_object_land_access_enabled()',
 'atol()',
 'atomic_add()',
 'board_init_kernel_timer()',
 'bulk_seek_f()',
 'bzero()',
 'calloc()',
 'checkRange()',
 'create_pool()',
 'dbg_add_command()',
 'disk_page_cache_modified()',
 'disk_page_io_seek()',
 'exit()',
 'ftlbentry()',
 'get_current_thread()',
 'hal_cond_broadcast()',
 'hal_cond_init()',
 'hal_cond_signal()',
 'hal_cond_wait()',
 'hal_cpu_reset_real()',
 'hal_enable_softirq()',
 'hal_exit_kernel_thread()',
 'hal_init_object_vmem()',
 'hal_is_sti()',
 'hal_mem_pagesize()',
 'hal_mutex

In [44]:
# Worked well. Let's check also usys_* calls

# Reverting graph
kernel_main_subgraph_r = {}
for k, nodes in kernel_main_subgraph.items():
    for n in nodes:
        if not n in kernel_main_subgraph_r.keys():
            kernel_main_subgraph_r[n] = set()
        kernel_main_subgraph_r[n].add(k)

get_neighbours(kernel_main_subgraph_r, "usys_open()", 4)

{0: {'usys_open()'},
 1: {'k_open()'},
 4: {'load_classes_module()'},
 3: {'load_code()'},
 2: {'k_load_file()'}}

In [49]:
# Ok. Let's keep those functions
# Building list of used files

kernel_main_subgraph_nodes = set()

for k, nodes in kernel_main_subgraph.items():
    kernel_main_subgraph_nodes.add(k)
    kernel_main_subgraph_nodes.update(nodes)

len(kernel_main_subgraph_nodes)

364

In [55]:
kernel_main_subgraph_files = set()

for node in kernel_main_subgraph_nodes:
    if kernel_index.store[node].location:
        kernel_main_subgraph_files.add(kernel_index.store[node].location.split(":")[0])  

kernel_main_subgraph_files

{'/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/hal.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/ia32/pio.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/ia32/proc_reg.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/init.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/smp.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/pager_io_req.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/sys/libkern.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/sys/syslog.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/threads.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/boot_cmd_line.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/disk.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/disk_pool.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland

In [56]:
vm_files = ['/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/hal.h',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/threads.h',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/pager_io_req.h',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/snap_sync.c',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/smp.h',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/pagelist.h',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/fsck.c',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/pager.c',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/disk.c',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/user_trap.c',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/hal.c',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/vm_connect.c',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/vm_load.c',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/hal_physmem.c',
             '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/vm_map.c']

In [57]:
set(kernel_main_subgraph_files).intersection(set(vm_files))

{'/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/hal.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/smp.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/pager_io_req.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/threads.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/disk.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/fsck.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/hal.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/hal_physmem.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/pagelist.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/pager.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/snap_sync.c',
 '/

In [58]:
len(set(kernel_main_subgraph_files).intersection(set(vm_files)))

15

In [59]:
len(vm_files)

16

In [60]:
set(vm_files) - set(kernel_main_subgraph_files)

{'/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/vm_connect.c'}

In [61]:
set(kernel_main_subgraph_files) - set(vm_files)

{'/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/ia32/pio.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/ia32/proc_reg.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/init.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/sys/libkern.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/sys/syslog.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/boot_cmd_line.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/disk_pool.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/dpc.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/driver_pci_intel_etc.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/fs_map.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/heap.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/

In [63]:
# Attention to pio.h and proc_reg.h
# Probably, both can be omitted. But let's check how pio.h is used

for f_name, node in kernel_index.store.items():
    if node.location and node.location.split(":")[0].endswith("pio.h"):
        print("--- " + f_name)
        print(kernel_index.get_callers(f_name))

--- __outbc()
['get_fb_seg()', 'read_vga_register()', 'parallels_video_start()', 'cirrus_dump_cursol_palette()', 'ne_ei()', 'ne_reset()', 'ne_probe()', 'sub_writeBusMstrStatus()', 'isa_write8()', 'int_handler()', 'write_vga_register()', 'sub_writeBusMstrCmd()', 'sregs()', 'cirrus_unlock()', 'rtl8139_stop()', 'pio_outbyte()', 'virtio_reset()', 'cirrus_load_cursol_palette()', 'eth_pio_read()', 'eth_pio_write()', 'ne_interrupt()', 'rtl8139_resetrx()', 'setPlane()', 'fubyte()', 'setpal()', 'ne_poll()', 'virtio_set_status()', 'virtio_set_config_struct()', 'virtio_notify()', 'hal_PIC_interrupt_dispatcher()', 'debug_console_putc()', 'ne_probe0()', 'AcpiOsWritePort()', 'enable_multicast()', 'parallels_video_stop()', 'rtl8139_init()', 'arch_debug_console_init()', 'SetMode()']
--- __outb()
['get_fb_seg()', 'read_vga_register()', 'parallels_video_start()', 'cirrus_dump_cursol_palette()', 'ne_ei()', 'ne_reset()', 'ne_probe()', 'sub_writeBusMstrStatus()', 'isa_write8()', 'int_handler()', 'write_vga

In [65]:
# Ok, let's check how it is used inside the subgraph

for f_name, node in kernel_index.store.items():
    if f_name in kernel_main_subgraph_nodes and node.location and node.location.split(":")[0].endswith("pio.h"):
        print("--- " + f_name)
        print(get_neighbours(kernel_main_subgraph_r, f_name, 2))


--- __outwc()
{0: {'__outwc()'}, 1: {'ia32_intel_poweroff()'}, 2: {'phantom_shutdown()'}}
--- __outw()
{2: {'phantom_shutdown()'}, 1: {'ia32_intel_poweroff()'}, 0: {'__outw()'}}
--- __inw()
{0: {'__inw()'}, 2: {'phantom_shutdown()'}, 1: {'ia32_intel_poweroff()'}}
--- __inwc()
{2: {'phantom_shutdown()'}, 1: {'ia32_intel_poweroff()'}, 0: {'__inwc()'}}


In [71]:
# Ok, nothing scary :)
# Let's filter them and then copy to isomem folder

sorted([x[len("/home/anton/Sandbox/PhantomOS/test/phantomuserland/"):] for x in set(kernel_main_subgraph_files) - set(vm_files) if not "ia32" in x and not "include" in x])

['oldtree/kernel/phantom/boot_cmd_line.c',
 'oldtree/kernel/phantom/disk_pool.c',
 'oldtree/kernel/phantom/dpc.c',
 'oldtree/kernel/phantom/driver_pci_intel_etc.c',
 'oldtree/kernel/phantom/fs_map.c',
 'oldtree/kernel/phantom/heap.c',
 'oldtree/kernel/phantom/ide_io.c',
 'oldtree/kernel/phantom/intrdisp.c',
 'oldtree/kernel/phantom/net_timer.c',
 'oldtree/kernel/phantom/test_switch.c',
 'oldtree/kernel/phantom/time.c',
 'oldtree/kernel/phantom/timedcall.c']

In [77]:
# Probably, we can use everything except ide_io since it will be replaced
# Also, time.c looks not really good. Let's check what is used

# Ok, let's check how it is used inside the subgraph

for f_name, node in kernel_index.store.items():
    if f_name in kernel_main_subgraph_nodes and node.location and node.location.split(":")[0].endswith("time.c"):
        print("--- " + f_name)
#         print(get_neighbours(kernel_main_subgraph_r, f_name, 2))

--- hal_time_init()
--- arch_get_tick_rate()
--- arch_get_time_delta()
